# VOCS v1 data structure 

In [1]:
from xopt.vocs import VOCS, ObjectiveEnum, ConstraintEnum

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == ObjectiveEnum.MAXIMIZE

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

In [10]:
vocs.schema()

{'title': 'VOCS',
 'description': 'Variables, Objectives, Constraints, and other Settings (VOCS) data structure\nto describe optimization problems.',
 'type': 'object',
 'properties': {'variables': {'title': 'Variables',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'type': 'number'},
    'minItems': 2,
    'maxItems': 2}},
  'constraints': {'title': 'Constraints',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'array',
    'items': {'anyOf': [{'type': 'number'},
      {'$ref': '#/definitions/ConstraintEnum'}]},
    'minItems': 2,
    'maxItems': 2}},
  'objectives': {'default': {},
   'type': 'object',
   'additionalProperties': {'$ref': '#/definitions/ObjectiveEnum'}},
  'constants': {'title': 'Constants', 'default': {}, 'type': 'object'},
  'linked_variables': {'title': 'Linked Variables',
   'default': {},
   'type': 'object',
   'additionalProperties': {'type': 'string'}}},
 'additionalProperties': Fal

# Objective Evaluation

In [11]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,702.048291,-0.359981,1234,701.688309,702.408272,1403.376618,1404.816544
6,61.732734,-0.949709,1234,60.783025,62.682443,121.566050,125.364886
7,489.559945,0.356630,1234,489.916575,489.203314,979.833150,978.406628
8,588.441182,-0.428304,1234,588.012878,588.869486,1176.025755,1177.738973
9,971.584613,0.053483,1234,971.638096,971.531130,1943.276192,1943.062260
10,947.267522,0.013172,1234,947.280695,947.254350,1894.561390,1894.508700
11,729.872330,0.739050,1234,730.611380,729.133279,1461.222760,1458.266559
12,405.120070,0.628362,1234,405.748432,404.491708,811.496865,808.983416
13,620.283133,0.120142,1234,620.403275,620.162991,1240.806550,1240.325983
14,842.643505,-0.083246,1234,842.560259,842.726750,1685.120518,1685.453500


In [12]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [13]:
?form_objective_data

Signature: form_objective_data(objectives: Dict, data, prefix='objective_')
Docstring:
Use objective dict and data (dataframe) to generate objective data (dataframe)

Weights are applied to convert all objectives into mimimization form.

Returns a dataframe with the objective data intented to be minimized.

Missing or nan values will be filled with: np.inf
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [14]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-701.688309,702.408272
6,-60.783025,62.682443
7,-489.916575,489.203314
8,-588.012878,588.869486
9,-971.638096,971.531130
10,-947.280695,947.254350
11,-730.611380,729.133279
12,-405.748432,404.491708
13,-620.403275,620.162991
14,-842.560259,842.726750


In [15]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-701.688309,702.408272
6,-60.783025,62.682443
7,-489.916575,489.203314
8,-588.012878,588.869486
9,-971.638096,971.531130
10,-947.280695,947.254350
11,-730.611380,729.133279
12,-405.748432,404.491708
13,-620.403275,620.162991
14,-842.560259,842.726750


In [16]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-701.68830918,  702.40827189],
       [ -60.78302515,   62.68244277],
       [-489.91657502,  489.20331421],
       [-588.01287769,  588.86948637],
       [-971.63809624,  971.53112979],
       [-947.28069477,  947.25435013],
       [-730.61138001,  729.13327941],
       [-405.74843232,  404.49170781],
       [-620.40327513,  620.16299144],
       [-842.56025898,  842.72675022]])

In [17]:
?form_constraint_data

Signature:
form_constraint_data(
    constraints: Dict,
    data: pandas.core.frame.DataFrame,
    prefix='constraint_',
)
Docstring:
Use constraint dict and data (dataframe) to generate constraint data (dataframe). A
constraint is satisfied if the evaluation is < 0.

Args:
    constraints: Dictonary of constraints
    data: Dataframe with the data to be evaluated
    prefix: Prefix to use for the transformed data in the dataframe

Returns a dataframe with the constraint data.

Missing or nan values will be filled with: np.inf
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [18]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1401.376618,-1404.816544
6,119.566050,-125.364886
7,977.833150,-978.406628
8,1174.025755,-1177.738973
9,1941.276192,-1943.062260
10,1892.561390,-1894.508700
11,1459.222760,-1458.266559
12,809.496865,-808.983416
13,1238.806550,-1240.325983
14,1683.120518,-1685.453500


In [19]:
?form_feasibility_data

Signature: form_feasibility_data(constraints: Dict, data, prefix='feasible_')
Docstring:
Use constraint dict and data to identify feasible points in the the dataset.

Returns a dataframe with the feasibility data.
File:      ~/Code/GitHub/Xopt/xopt/vocs.py
Type:      function


In [20]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [21]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [22]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [23]:
data['a']  = np.nan

In [24]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [25]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [26]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
